In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Problem Statement

**Data Analysis case study**

I am working on a logistic problem where they have faced cancellation of orders in their deliveries. 
Their main goal is to be the 1st prefered delivery partner in Africa.

**Challenge 2**

-There's been deliberate deletion of data
1. Use the data 2, missed data to predict cancellation reasons for all orders with missing data.
2. Document steps and method used.

How to predict cancellation reasons
Our algorithm will be a multi-class classification machine learning algorithm which involves the following steps:

1. Importing libraries
2. Fetching the dataset
3. Creating the dependent variable class
4. Extracting features and output
5. Train-Test dataset splitting (include test dataset as holdout data)
6. Feature scaling
7. Training the model
8. Calculating the model score using the metric deemed fit based on the problem
9. Saving the model for predicting cancellation reasons on the test dataset

In [103]:
##for faster data processing
import numba as nb
import dask as da

#for data manipulation 
import pandas as pd
import numpy as np

##for data visualizations, distribution  and prediction of the variables
from pandas_summary import DataFrameSummary
import scipy as sc
import sklearn as skl #used for splitting data into test and train and training the random forest.

import seaborn as sns

import random

# Import Dataset

In [138]:
# I’ll start by using pandas.read_csv() library to read both files 
query_df = pd.read_csv('../Sendy_data/prediction_df.csv', sep=',')
test_df = pd.read_csv('../Sendy_data/test_df.csv', sep=',')


In [139]:
##Show the first five records of the dataset.
query_df.head()

,index,personal_or_business,business_number,vehicle_type,standard_or_express,platform,cancel_reason,cancelled_by,placement_day,placement_time,...,distance_(km),pickup_lat,pickup_long,scheduled,order_timedelta,processing_timedelta,confirmation_timedelta,order_period,processing_period,confirmation_period
0,0,Personal,-1.0,Bike,Express,android,No driver has been allocated to my request.,Customer,1,2019-07-29 17:55:38,...,9,-1.313053,36.841437,OnDemAND,NaN,-1 days +23:59:03.000000000,NaN,0,0,0
1,1,Personal,-1.0,Bike,Express,android,My order is not ready.,Customer,1,2019-07-29 12:26:27,...,14,-1.318959,36.853394,OnDemAND,-1 days +23:58:51.000000000,-1 days +23:57:43.000000000,-1 days +23:58:52.000000000,0,0,0
2,2,Business,20680.0,Bike,Express,android,I placed the wrong locations.,Customer,1,2019-07-29 13:21:18,...,16,-1.313854,36.706867,OnDemAND,-1 days +23:59:44.000000000,-1 days +23:33:49.000000000,-1 days +23:34:05.000000000,0,0,0
3,3,Personal,-1.0,Bike,Express,android,My order is not ready.,Customer,1,2019-07-29 10:29:19,...,20,-1.206197,36.778776,OnDemAND,NaN,-1 days +23:59:50.000000000,NaN,0,0,0
4,4,Personal,-1.0,Bike,Standard,ios,No Partner,Staff,1,2019-07-29 14:34:05,...,11,-1.296958,36.808166,OnDemAND,-1 days +23:34:57.000000000,-1 days +22:44:24.000000000,-1 days +23:09:27.000000000,0,0,0


In [140]:
##Show the first five records of the dataset.
test_df.head()

,personal_or_business,business_number,vehicle_type,standard_or_express,platform,cancel_reason,cancelled_by,placement_day,placement_time,confirmation_day,confirmation_time,cancellation_day,cancellation_time,distance_(km),pickup_lat,pickup_long,scheduled
0,Personal,-1.0,Bike,Express,ios,NaN,Customer,1,15:23:44,1,16:40:47,1,16:43:50,4,-1.299370,36.822107,OnDemAND
1,Business,20680.0,Bike,Express,desktop,NaN,Staff,1,14:21:46,1,14:22:20,1,14:30:48,17,-1.264490,36.789303,OnDemAND
2,Business,17968.0,Bike,Express,desktop,NaN,Customer,1,15:13:20,1,NaN,1,15:56:34,18,-1.427056,36.961499,OnDemAND
3,Personal,-1.0,Bike,Standard,android,NaN,Customer,1,13:54:19,1,NaN,1,13:56:41,8,-1.301737,36.743936,OnDemAND
4,Personal,-1.0,Bike,Standard,android,NaN,Staff,1,13:24:05,1,15:57:20,1,17:14:46,13,-1.308098,36.920464,Scheduled


In [142]:
##check the rows and the columns
query_df.shape

(6393, 24)

## Creating the dependent variable class for the query dataframe

This is basically converting all the column values from string to intergers because the algorithms can only read numbers. 

In [144]:
# traversing through the column values of dataFrame and writing values where  condition matches. 
query_df['personal_or_business'][query_df['personal_or_business'] == 'Personal'] = 1
query_df['personal_or_business'][query_df['personal_or_business'] == 'Business'] = 2

query_df['vehicle_type'][query_df['vehicle_type'] == 'Bike'] = 1
query_df['vehicle_type'][query_df['vehicle_type'] == 'Standard Bike'] = 2

query_df['standard_or_express'][query_df['standard_or_express'] == 'Express'] = 1
query_df['standard_or_express'][query_df['standard_or_express'] == 'Standard'] = 2

query_df['cancelled_by'][query_df['cancelled_by'] == 'Customer'] = 1
query_df['cancelled_by'][query_df['cancelled_by'] == 'Staff'] = 2

query_df['scheduled'][query_df['scheduled'] == 'OnDemAND'] = 1
query_df['scheduled'][query_df['scheduled'] == 'Scheduled'] = 2

query_df['platform'][query_df['platform'] == 'android'] = 1
query_df['platform'][query_df['platform'] == 'ios'] = 2
query_df['platform'][query_df['platform'] == 'desktop'] = 3
query_df['platform'][query_df['platform'] == 'API'] = 4
query_df['platform'][query_df['platform'] == 'peer_web'] = 5


/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/dancilla/PYTHON_37/lib/pyth

Use one hot encoding for the cancel reason values

In [145]:
from sklearn.preprocessing import label_binarize

#Transform labels into one hot encoded format.
y_train_ohe = label_binarize(query_df['cancel_reason'], classes=['No driver has been allocated to my request.', 
                                                                 'My order is not ready.', 'I placed the wrong locations.',
                                                                'No Partner', 'Customer error', 'The driver is too far away.'
                                                                'Delay'])
print('y_train_ohe shape: {}'.format(y_train_ohe.shape))
print('y_train_ohe samples:')
print(y_train_ohe[:5])

y_train_ohe shape: (6393, 6)
y_train_ohe samples:
[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]]


In [146]:
query_df.head()

,index,personal_or_business,business_number,vehicle_type,standard_or_express,platform,cancel_reason,cancelled_by,placement_day,placement_time,...,distance_(km),pickup_lat,pickup_long,scheduled,order_timedelta,processing_timedelta,confirmation_timedelta,order_period,processing_period,confirmation_period
0,0,1,-1.0,1,1,1,No driver has been allocated to my request.,1,1,2019-07-29 17:55:38,...,9,-1.313053,36.841437,1,NaN,-1 days +23:59:03.000000000,NaN,0,0,0
1,1,1,-1.0,1,1,1,My order is not ready.,1,1,2019-07-29 12:26:27,...,14,-1.318959,36.853394,1,-1 days +23:58:51.000000000,-1 days +23:57:43.000000000,-1 days +23:58:52.000000000,0,0,0
2,2,2,20680.0,1,1,1,I placed the wrong locations.,1,1,2019-07-29 13:21:18,...,16,-1.313854,36.706867,1,-1 days +23:59:44.000000000,-1 days +23:33:49.000000000,-1 days +23:34:05.000000000,0,0,0
3,3,1,-1.0,1,1,1,My order is not ready.,1,1,2019-07-29 10:29:19,...,20,-1.206197,36.778776,1,NaN,-1 days +23:59:50.000000000,NaN,0,0,0
4,4,1,-1.0,1,2,2,No Partner,2,1,2019-07-29 14:34:05,...,11,-1.296958,36.808166,1,-1 days +23:34:57.000000000,-1 days +22:44:24.000000000,-1 days +23:09:27.000000000,0,0,0


### Change timelines to pandas datetime

In [147]:
query_df['placement_time'] = pd.to_datetime(query_df['placement_time'])
query_df['confirmation_time'] = pd.to_datetime(query_df['confirmation_time'])
query_df['cancellation_time'] = pd.to_datetime(query_df['cancellation_time'])


## Split data

In [148]:
query_df.keys()

Index(['index', 'personal_or_business', 'business_number', 'vehicle_type',
       'standard_or_express', 'platform', 'cancel_reason', 'cancelled_by',
       'placement_day', 'placement_time', 'confirmation_day',
       'confirmation_time', 'cancellation_day', 'cancellation_time',
       'distance_(km)', 'pickup_lat', 'pickup_long', 'scheduled',
       'order_timedelta', 'processing_timedelta', 'confirmation_timedelta',
       'order_period', 'processing_period', 'confirmation_period'],
      dtype='object')

In [149]:
test_df.keys()

Index(['personal_or_business', 'business_number', 'vehicle_type',
       'standard_or_express', 'platform', 'cancel_reason', 'cancelled_by',
       'placement_day', 'placement_time', 'confirmation_day',
       'confirmation_time', 'cancellation_day', 'cancellation_time',
       'distance_(km)', 'pickup_lat', 'pickup_long', 'scheduled'],
      dtype='object')

In [150]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib


#Choose the features which will be used for building the model
columns = ['personal_or_business', 'business_number', 'vehicle_type',
       'standard_or_express', 'platform', 'cancelled_by',
       'placement_day', 'confirmation_day', 'cancellation_day',
       'distance_(km)', 'pickup_lat', 'pickup_long', 'scheduled']


X = query_df[columns]
y = y_train_ohe

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=0)

**In this case, since we are predicting multiple values, we need to use the multiclass classification using
using Random Forest on Scikit-Learn Library**


## Feature Scaling

In [151]:
# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/home/dancilla/PYTHON_37/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


## Training the model

### Apply  Random forest classifier to Test Data

In [163]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score 

# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_estimators=300, max_depth=2, random_state=0)

# Train the Classifier to take the training features and learn how they relate to the training y (the cancel reasons)
clf.fit(X_train, y_train)  

# Fitting Random Forest Classification to the Training set
prediction=clf.predict(X_test)
print('The accuracy of the Random Classifier is', metrics.accuracy_score(prediction,y_test))

The accuracy of the Random Classifier is 0.15715402658326819


In [164]:
# View the predicted probabilities of the first 10 observations
clf.predict_proba(X_test)[0:10]

[array([[0.69302968, 0.30697032],
        [0.6936686 , 0.3063314 ],
        [0.66770816, 0.33229184],
        ...,
        [0.66312643, 0.33687357],
        [0.81229544, 0.18770456],
        [0.65231768, 0.34768232]]), array([[0.74243853, 0.25756147],
        [0.7482793 , 0.2517207 ],
        [0.78422106, 0.21577894],
        ...,
        [0.78206502, 0.21793498],
        [0.88986096, 0.11013904],
        [0.79333771, 0.20666229]]), array([[0.77282804, 0.22717196],
        [0.77505724, 0.22494276],
        [0.79971957, 0.20028043],
        ...,
        [0.79913701, 0.20086299],
        [0.90099577, 0.09900423],
        [0.81114441, 0.18885559]]), array([[0.89775607, 0.10224393],
        [0.89891052, 0.10108948],
        [0.90508842, 0.09491158],
        ...,
        [0.91371188, 0.08628812],
        [0.59123892, 0.40876108],
        [0.90791719, 0.09208281]]), array([[0.97541644, 0.02458356],
        [0.97566284, 0.02433716],
        [0.97799497, 0.02200503],
        ...,
        [0.98

## evaluation of classification perfomance

**Now that we have predicted the cancel reasons in the test data, we can compare our predicted cancel  reasons with the that order actual reasons.**

In [165]:
#check the perfomance of the model by using f1_score, precision_score, recall_score, 
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

print ('The precision score of the Random Classifier is', precision_score(y_test, prediction, average='weighted'))
print ('The recall score of the Random Classifier is', recall_score(y_test, prediction, average='weighted'))
print ('The F1 score of the Random Classifier is', f1_score(y_test, prediction, average='weighted'))



The precision score of the Random Classifier is 0.16780198540036406
The recall score of the Random Classifier is 0.059336823734729496
The F1 score of the Random Classifier is 0.08767182383277448


/home/dancilla/PYTHON_37/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### View Feature Importance

**This gives a score telling us how important each feature was in classifying. We can clearly see thatsome features were more important in classification than others.**

In [166]:
# View a list of the features and their importance scores
list(zip(X, clf.feature_importances_))

[('personal_or_business', 0.029037254042804418),
 ('business_number', 0.07170065994243856),
 ('vehicle_type', 0.0),
 ('standard_or_express', 0.018156057299197635),
 ('platform', 0.20289983696334435),
 ('cancelled_by', 0.4294071080779935),
 ('placement_day', 0.003212169094485514),
 ('confirmation_day', 0.003364760740313939),
 ('cancellation_day', 0.004183909447310197),
 ('distance_(km)', 0.15537445054445553),
 ('pickup_lat', 0.03729793974518505),
 ('pickup_long', 0.03276497426190171),
 ('scheduled', 0.012600879840569689)]

**Store the Random Forest classifier using the joblib function of sklearn.**

## Holdout

In [156]:
holdout = test_df # from now on we will refer to this
               # dataframe as the holdout data
holdout.columns

Index(['personal_or_business', 'business_number', 'vehicle_type',
       'standard_or_express', 'platform', 'cancel_reason', 'cancelled_by',
       'placement_day', 'placement_time', 'confirmation_day',
       'confirmation_time', 'cancellation_day', 'cancellation_time',
       'distance_(km)', 'pickup_lat', 'pickup_long', 'scheduled'],
      dtype='object')

In [157]:
holdout.head()

,personal_or_business,business_number,vehicle_type,standard_or_express,platform,cancel_reason,cancelled_by,placement_day,placement_time,confirmation_day,confirmation_time,cancellation_day,cancellation_time,distance_(km),pickup_lat,pickup_long,scheduled
0,Personal,-1.0,Bike,Express,ios,NaN,Customer,1,15:23:44,1,16:40:47,1,16:43:50,4,-1.299370,36.822107,OnDemAND
1,Business,20680.0,Bike,Express,desktop,NaN,Staff,1,14:21:46,1,14:22:20,1,14:30:48,17,-1.264490,36.789303,OnDemAND
2,Business,17968.0,Bike,Express,desktop,NaN,Customer,1,15:13:20,1,NaN,1,15:56:34,18,-1.427056,36.961499,OnDemAND
3,Personal,-1.0,Bike,Standard,android,NaN,Customer,1,13:54:19,1,NaN,1,13:56:41,8,-1.301737,36.743936,OnDemAND
4,Personal,-1.0,Bike,Standard,android,NaN,Staff,1,13:24:05,1,15:57:20,1,17:14:46,13,-1.308098,36.920464,Scheduled


## Creating the dependent variable class on the holdout dataset

In [158]:
# traversing through personal or business orders 
# column of dataFrame and writing values where  condition matches. 
holdout['personal_or_business'][holdout['personal_or_business'] == 'Personal'] = 1
holdout['personal_or_business'][holdout['personal_or_business'] == 'Business'] = 2

holdout['standard_or_express'][holdout['standard_or_express'] == 'Express'] = 1
holdout['standard_or_express'][holdout['standard_or_express'] == 'Standard'] = 2

holdout['cancelled_by'][holdout['cancelled_by'] == 'Customer'] = 1
holdout['cancelled_by'][holdout['cancelled_by'] == 'Staff'] = 2

holdout['vehicle_type'][holdout['vehicle_type'] == 'Bike'] = 1
holdout['vehicle_type'][holdout['vehicle_type'] == 'Standard Bike'] = 2

holdout['scheduled'][holdout['scheduled'] == 'OnDemAND'] = 1
holdout['scheduled'][holdout['scheduled'] == 'Scheduled'] = 2


holdout['platform'][holdout['platform'] == 'android'] = 1
holdout['platform'][holdout['platform'] == 'ios'] = 2
holdout['platform'][holdout['platform'] == 'desktop'] = 3
holdout['platform'][holdout['platform'] == 'API'] = 4
holdout['platform'][holdout['platform'] == 'peer_web'] = 5


/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  

In [159]:
#Choose the features which will be used for building the model
columns = ['personal_or_business', 'business_number', 'vehicle_type',
       'standard_or_express', 'platform', 'cancelled_by',
       'placement_day', 'confirmation_day', 'cancellation_day',
       'distance_(km)', 'pickup_lat', 'pickup_long', 'scheduled']
holdout=holdout[columns]

## Making predictions on unseen data

**Generate predictions using .predict_proba() on the holdout columns.**


In [32]:
# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

# Train the Classifier to take the training features and learn how they relate to the training y (the cancel reasons)
clf.fit(X_train, y_train) 
holdout_predictions = clf.predict_proba(holdout[columns])

## Creating a submission file


In [160]:
#Create a submission file using the holdout_predictions, to show the predictions done.
submission_df = {"cancel_reason": holdout_predictions}
submission = pd.DataFrame(submission_df)

In [161]:
submission.to_csv("cancelreason.csv",index=False)